In [ ]:
!pip install selenium webdriver-manager

In [ ]:
import time
import csv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

def extrair_tudo_saude_v2():
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

    base_url = "https://www.gov.br/saude/pt-br/assuntos/saude-de-a-a-z"
    letras = "abcdefghijklmnopqrstuvwxyz"
    arquivo_nome = 'saude_brasil_completo.csv'

    with open(arquivo_nome, 'w', newline='', encoding='utf-8-sig') as f:
        escritor = csv.writer(f)
        escritor.writerow(['Letra', 'Tema', 'URL', 'Conteudo_Completo'])

        print("--- INICIANDO VARREDURA COMPLETA ---")

        for letra in letras:
            url_letra = f"{base_url}/{letra}"
            print(f"\n--- Letra {letra.upper()} ---")

            try:
                driver.get(url_letra)
                time.sleep(3)

                # Passo 1: Captura links da letra
                links_elementos = driver.find_elements(By.TAG_NAME, "a")
                urls_da_letra = []
                for el in links_elementos:
                    href = el.get_attribute("href")
                    texto = el.text.strip()
                    if href and f"/saude-de-a-a-z/{letra}/" in href and len(texto) > 3:
                        if href not in [u[1] for u in urls_da_letra]:
                            urls_da_letra.append((texto, href))

                print(f"Encontrados {len(urls_da_letra)} temas.")

                # Passo 2: Extração profunda de cada tema
                for nome_tema, url_tema in urls_da_letra:
                    try:
                        driver.get(url_tema)

                        # Aguarda até 10 segundos para o corpo do texto aparecer
                        wait = WebDriverWait(driver, 10)

                        # Tenta vários seletores conhecidos do portal Gov.br
                        seletores = [
                            "div#parent-fieldname-text",
                            "div#content-core",
                            "article.tipo-topico",
                            ".prose"
                        ]

                        conteudo_texto = ""
                        for seletor in seletores:
                            try:
                                elemento = driver.find_element(By.CSS_SELECTOR, seletor)
                                if elemento.text.strip():
                                    conteudo_texto = elemento.text
                                    break
                            except:
                                continue

                        if not conteudo_texto:
                            # Fallback: pega todos os parágrafos se os IDs falharem
                            ps = driver.find_elements(By.TAG_NAME, "p")
                            conteudo_texto = " ".join([p.text for p in ps])

                        texto_limpo = " ".join(conteudo_texto.split())

                        if len(texto_limpo) > 10:
                            escritor.writerow([letra.upper(), nome_tema, url_tema, texto_limpo])
                            print(f"  > {nome_tema}: OK")
                        else:
                            print(f"  > {nome_tema}: Conteúdo muito curto/vazio")

                    except Exception as e:
                        print(f"  > Erro ao ler: {nome_tema}")
                        continue

            except Exception as e:
                print(f"Falha na letra {letra.upper()}")

    driver.quit()
    print(f"\nConcluído! Arquivo: {arquivo_nome}")

if __name__ == "__main__":
    extrair_tudo_saude_v2()

--- INICIANDO VARREDURA COMPLETA ---

--- Letra A ---
Encontrados 12 temas.
  > Acidentes por animais peçonhentos: OK
  > Aedes aegypti: OK
  > HIV/Aids: OK
  > Amamentação: OK
  > Alzheimer: OK
  > Anomalias Congênitas: OK
  > Arboviroses: OK
  > Arenavírus: OK
  > Asma: OK
  > Aspergilose: OK
  > Atrofia Muscular Espinhal: OK
  > Autismo: OK

--- Letra B ---
Encontrados 5 temas.
  > Biotinidase (deficiência): OK
  > Botulismo: OK
  > Bronquiolite: OK
  > Bronquite: OK
  > Brucelose Humana: OK

--- Letra C ---
Encontrados 18 temas.
  > Calendário da Saúde: OK
  > Calor: OK
  > Candidíase sistêmica: OK
  > Catapora (Varicela): OK
  > Caxumba: OK
  > Chikungunya: OK
  > Coccidioidomicose: OK
  > Cólera: OK
  > Coqueluche: OK
  > Covid-19: OK
  > Criptococose: OK
  > Cromoblastomicose: OK
  > Câncer: OK
  > Câncer de boca: OK
  > Câncer de mama: OK
  > Câncer de pele: OK
  > Câncer de pênis: OK
  > Câncer de próstata: OK

--- Letra D ---
Encontrados 17 temas.
  > Dengue: OK
  > Depressão

In [ ]:
from google.colab import files
files.download('saude_brasil_completo.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import csv

def converter_csv_para_md(arquivo_entrada, arquivo_saida, coluna_texto):
    try:
        with open(arquivo_entrada, mode='r', encoding='utf-8') as csv_file:
            # O reader do Python já remove as aspas automaticamente
            leitor_csv = csv.DictReader(csv_file)

            with open(arquivo_saida, mode='w', encoding='utf-8') as md_file:
                md_file.write("# Dados de Treinamento SAC\n\n")

                for linha in leitor_csv:
                    texto = linha[coluna_texto]
                    # Escreve o texto no formato de parágrafos ou lista no Markdown
                    md_file.write(f"### Informação\n{texto}\n\n---\n\n")

        print(f"Sucesso! Arquivo {arquivo_saida} gerado com sucesso.")

    except Exception as e:
        print(f"Erro ao converter: {e}")

# Use assim:
# nome_do_arquivo_csv, nome_do_arquivo_saida, nome_da_coluna_dentro_do_csv
converter_csv_para_md('saude_brasil_completo.csv', 'dados-sac.md', 'Conteudo_Completo')

Sucesso! Arquivo dados-sac.md gerado com sucesso.
